In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from traitlets import link

In [2]:
center = [34.6252978589571, -77.34580993652344]
# Boca Chica
center = [25.988338, -97.184157]
zoom = 13

In [3]:
m = Map(center=center, zoom=zoom)
m

Map(center=[25.988338, -97.184157], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

created
{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#0000FF', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-97.212235, 25.964775], [-97.212235, 25.990711], [-97.159175, 25.990711], [-97.159175, 25.964775], [-97.212235, 25.964775]]]}}


In [4]:
m.zoom = 13.0
m.center = [25.988338, -97.184157]
#m.center = [27.0, -97.184157]

Now create the `DrawControl` and add it to the `Map` using `add_control`. We also register a handler for draw events. This will fire when a drawn path is created, edited or deleted (there are the actions). The  `geo_json` argument is the serialized geometry of the drawn path, along with its embedded style.

In [5]:
dc = DrawControl(marker={'shapeOptions': {'color': '#0000FF'}},
                 rectangle={'shapeOptions': {'color': '#0000FF'}},
                 circle={'shapeOptions': {'color': '#0000FF'}},
                 circlemarker={},
                 )

def handle_draw(target, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)

In [30]:
print(m.bounds,m.center,m.crs,m.east)
print(dir(m.comm))
print(m.layers)

((25.95742729855581, -97.26831436157228), (26.01914873523295, -97.09991455078126)) [25.988338, -97.184157] {'name': 'EPSG3857', 'custom': False} -97.09991455078126
['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_notifiers', '_close_callback', '_close_data', '_closed', '_config_changed', '_cross_validation_lock', '_default_comm_id', '_default_kernel', '_default_topic', '_defining_class', '_find_my_config', '_get_log_handler', '_get_trait_default_generator', '_load_config', '_log_default', '_msg_callback', '_notify_observers', '_notify_trait', '_open_data', '_publish_msg', '_register_validator', '_remove_notifiers', '_trait_default_generators', '_t

In addition, the `DrawControl` also has `last_action` and `last_draw` attributes that are created dynamicaly anytime a new drawn path arrives.

In [7]:
dc.last_action

''

In [8]:
dc.last_draw

{'type': 'Feature', 'geometry': None}

It's possible to remove all drawings from the map

In [9]:
dc.clear_circles()

In [10]:
dc.clear_polylines()

In [11]:
dc.clear_rectangles()

In [12]:
dc.clear_markers()

In [13]:
dc.clear_polygons()

In [14]:
dc.clear()

Let's draw a second map and try to import this GeoJSON data into it.

In [15]:
m2 = Map(center=center, zoom=zoom, layout=dict(width='600px', height='400px'))
m2

Map(center=[25.988338, -97.184157], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

We can use `link` to synchronize traitlets of the two maps:

In [16]:
map_center_link = link((m, 'center'), (m2, 'center'))
map_zoom_link = link((m, 'zoom'), (m2, 'zoom'))

In [17]:
new_poly = GeoJSON(data=dc.last_draw)

In [18]:
m2.add_layer(new_poly)

Note that the style is preserved! If you wanted to change the style, you could edit the `properties.style` dictionary of the GeoJSON data. Or, you could even style the original path in the `DrawControl` by setting the `polygon` dictionary of that object. See the code for details.

Now let's add a `DrawControl` to this second map. For fun we will disable lines and enable circles as well and change the style a bit.

In [19]:
dc2 = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={},
                  circle={'shapeOptions': {'color': '#0000FF'}})
m2.add_control(dc2)